In [1]:
from joblib import load
import numpy as np
import pandas as pd
import re

In [2]:
def predict(data):
    encoder = load('encoder.joblib')
    scaler = load('scaler.joblib')
    etree = load('etree.joblib')
    
    cat_columns=['name','fuel','seller_type','owner']
    data[cat_columns]= encoder.transform(data[cat_columns])
    
    num_columns=['km_driven','mileage','engine','max_power','torque_NM','torque_rpm']
    data[num_columns]= scaler.transform(data[num_columns])
    
    return etree.predict(data)

#### Testing the model with pywebio

In [3]:
from pywebio.input import input,TEXT,radio,select,FLOAT,NUMBER
from pywebio.output import put_text

In [4]:
def validate_date(d):
    if d>2021 and d<1990:
        return 'Enter valid year'

In [5]:
seat_no = [2,4,5,6,7,8,9,10,14]
data = pd.DataFrame()

In [6]:
def predict_review():
    
    name = input("Enter the name of the Car：", type = TEXT)
    name = name.split(' ')[0]
    data['name'] = [name]
    
    year  = input("Enter the Year of purchase of the Car：", type = NUMBER, validate = validate_date)
    year = 2021 - year
    year = year / 2021
    data['year'] = year 
    
    km  = input("Enter the kilometer driven for the Car：", type = NUMBER)
    data['km_driven'] = km
    
    fuel_type = radio("Select the fuel type for the Car：", options = ['Diesel','Petrol','CNG','LPG'])
    data['fuel'] = fuel_type
    
    seller_type = radio("Select the seller type for the Car：", options = ['Individual','Dealer','Trustmark Dealer'])
    data['seller_type'] = seller_type
    
    transmission = radio("Select the gear type for the Car：", options = ['Manual','Automatic'])
    
    if transmission == 'Manual':
        transmission = 0
    else :
        transmission = 1
    
    data['transmission'] = transmission
    
    owner = radio("Select the owner type for the Car：", options = ['First Owner','Second Owner','Third Owner','Fourth & Above Owner','Test Drive Car'])
    data['owner'] = owner
    
    mileage  = input("Enter the mileage of the Car (kmpl)：", type = FLOAT)
    data['mileage'] = mileage
    
    engine  = input("Enter the engine of the Car (CC)：", type = NUMBER)
    data['engine'] = engine
    
    max_power  = input("Enter the max power of the Car (bhp)：", type = FLOAT)
    data['max_power'] = max_power
    
    seats = select("Select the number of seats for the Car：",seat_no)
    data['seats'] = seats
    
    torque = input("Enter the torque for the Car：", type = TEXT)
    
    def preprocess_torque(s):
        l = re.findall(r"\d*\.\d+|\d+", s)
        return ' '.join(l)
    
    torque = preprocess_torque(torque)
    
    torque_NM = float(torque.split(" ")[0])
    data['torque_NM'] = float(torque_NM)
    
    torque_rpm = float(torque.split(" ")[1])
    data['torque_rpm'] = float(torque_rpm)
    
    prediction = predict(data)
        
    put_text('Predicted Selling Price of the Car is :',round(prediction[0]))

In [7]:
predict_review()